# Model training for detecting vehicle type.

This model will be trained to recognize types of vehicle.

# Mounting Google Drive

Importing the drive module from the google.colab library to use the module's functions to interact with my Google Drive within the Colab notebook.

Mounting Google Drive to the specified directory (/content/drive) in the Colab environment to access the files and folders stored in my Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Ultralytics

Importing Ultralytics library, which is the official library for the YOLO (You Only Look Once) object detection models, in our case, we will be using YOLOv11.
It'll includes tools for our model training, validation, and inference.

In [2]:
# Installs the Ultralytics library

# > /dev/null ensures that the installation messages, such as progress bars and success notifications, are suppressed.
%pip install ultralytics > /dev/null

In [3]:
# Imports the Ultralytics library
import ultralytics

# Checks the current environment for compatibility and readiness to work with the Ultralytics library
ultralytics.checks()

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 32.6/235.7 GB disk)


# Hyperparameter Tuning (Grid Search)

Experimenting different batch sizes (8, 16, 32, 64, 128) with 2 different learning rates (0.01, 0.005)

In [4]:
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [0.01, 0.005]

In [5]:
from ultralytics import YOLO

In [6]:
# A UTF-8 locale is required.
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
for batch in batch_sizes:
    for lr in learning_rates:
        # YOLOv11 pre-trained model is reinitialized from the pre-trained weights (yolo11n.pt) at the start of each batch size experiment to test each batch size, learning rate combination independently
        model = YOLO("yolo11n.pt")
        print("\n#################################################")
        print(f"Training on batch size {batch} and learning rate {lr}")
        print("#################################################\n")

        # Define results directory in Google Drive
        results_dir = "/content/drive/My Drive/vehicle-type-detect-results"
        run_name = f"bs{batch}_lr{lr}"
        val_name = f"val_bs{batch}_lr{lr}"

        train_result = model.train(
            data="/content/drive/My Drive/vehicle-cv-1/data.yaml",
            epochs=100,
            imgsz=640,
            device="gpu",
            batch=batch,            # Adjust batch size
            lr0=lr,                 # Adjust learning rate
            project=results_dir,    # Save directly to Drive folder
            name=run_name,          # Each combination gets its own folder
        )
        model.val(
            project=results_dir,
            name=val_name
        )

100%|██████████| 5.35M/5.35M [00:00<00:00, 269MB/s]


#################################################
Training on batch size 8 and learning rate 0.01
#################################################

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:gpu (NVIDIA A100-SXM4-40GB, 40514MiB)


engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/drive/My Drive/vehicle-cv-1/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=gpu, workers=8, project=/content/drive/My Drive/vehicle-type-detect-results, name=bs8_lr0.01, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

100%|██████████| 755k/755k [00:00<00:00, 128MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

...

                   all         84       1873      0.609      0.398      0.423      0.267
                   bus         25         35      0.524      0.286      0.305      0.213
                  cars         84       1447      0.732      0.567      0.628      0.398
            motorcycle         61        188      0.664      0.309      0.347      0.161
                trucks         55        203      0.514      0.429      0.413      0.296
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/drive/My Drive/vehicle-type-detect-results/bs128_lr0.005
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:gpu (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11n summary (fused): 238 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/drive/My Drive/vehicle-cv-1/valid/labels.cache... 84 images, 0 backgrounds, 0 corrupt: 100%|██████████| 84/84 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 37, len(boxes) = 1873. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.91s/it]


                   all         84       1873      0.608      0.396      0.422      0.267
                   bus         25         35      0.525      0.286      0.302      0.212
                  cars         84       1447      0.732      0.566      0.627      0.398
            motorcycle         61        188      0.665      0.309      0.341       0.16
                trucks         55        203      0.511      0.424      0.416      0.299
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/drive/My Drive/vehicle-type-detect-results/val_bs128_lr0.005
